In [1]:
import gradio as gr
import os
import joblib


c:\Users\unaif\anaconda3\envs\projecto\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.chdir("../src")
from tool_preprocess import State_enc, product_encoder, sub_Issue_enc, sub_product_encoder,  Issue_enc, Company_response_enc, Time_enco,Company_enc, preprocesing_function_paraforest, week_enc
preprocesador = joblib.load("preprocesador_red.pkl")

c:\Users\unaif\anaconda3\envs\projecto\Lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\unaif\anaconda3\envs\projecto\Lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\unaif\anaconda3\envs\projecto\Lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\unaif\anaconda3\envs\projecto\Lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d

In [3]:
Time_enco

LabelEncoder()

In [4]:
# Aplicacion
import pandas as pd
import pickle
import os

import os
import joblib

os.chdir("../models")
with open('modelo_timely_tree_def.pkl', 'rb') as f:
    modelo_timely = pickle.load(f)

In [5]:
import dill
import __main__
import sys
sys.path.append(os.path.abspath("../src"))  # ajusta según tu estructura
def convert_to_str(X):
    return X.astype(str)
__main__.convert_to_str = convert_to_str



from tensorflow import keras
modelo_dispute = keras.models.load_model("modelo_dispute_red.keras")


In [6]:
product_encoder.classes_.tolist()

['Bank account or service',
 'Consumer loan',
 'Credit card',
 'Credit reporting',
 'Debt collection',
 'Money transfers',
 'Mortgage',
 'Other financial service',
 'Payday loan',
 'Prepaid card',
 'Student loan',
 'Unknown or not specified']

In [7]:
product_encoder.transform(pd.DataFrame(product_encoder.classes_.tolist()))

c:\Users\unaif\anaconda3\envs\projecto\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [8]:
import pickle
import joblib
os.chdir("../models")
#saco las pipelines /modelos


modelo_timely=joblib.load("modelo_timely_tree_def.pkl")
def predict_timely (    
    Product:str,
    Subproduct:str,
    Issue:str,
    Subissue:str,
    State:str,
    Companyresponse:str):
    
    #State por defecto Unknown
    #Subproduct por defecto Unknown or 
    #Subissue por defecto Unknown or 

    #Product por defecto other financia services

    #Yes=1, No= 0


    

    features=pd.DataFrame({"Product":[Product],"Sub-product": [Subproduct],	
            "Issue":[Issue],	
            "Sub-issue":[Subissue],	
            "State":[State],	
            "Company response":[Companyresponse]})

    display(features)
    features["Product"] = product_encoder.transform(features["Product"])
    features["Sub-product"] = sub_product_encoder.transform(features["Sub-product"])
    features["Issue"] = Issue_enc.transform(features["Issue"])
    features["Sub-issue"] = sub_Issue_enc.transform(features["Sub-issue"])
    features["State"] = State_enc.transform(features["State"])
    features["Company response"] = Company_response_enc.transform(features["Company response"])
    
    display(features)
    pred=modelo_timely.predict(features)[0]

    
    return {"response01": int(pred)}


In [9]:
predict_timely("Debt collection",	"Unknown or not specified",	"Cont'd attempts collect debt not owed",	"Debt is not mine"	,"TX"	,"Closed with non-monetary relief")

,Product,Sub-product,Issue,Sub-issue,State,Company response
0,Debt collection,Unknown or not specified,Cont'd attempts collect debt not owed,Debt is not mine,TX,Closed with non-monetary relief


,Product,Sub-product,Issue,Sub-issue,State,Company response
0,4,39,24,11,50,3


{'response01': 1}

In [10]:
pd.DataFrame(["Yes"])

,0
0,Yes


In [11]:
int(Time_enco.transform(pd.DataFrame(["Yes"]))[0])

c:\Users\unaif\anaconda3\envs\projecto\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


1

In [20]:
def predict_dispute (
    Product:int, 
    Subproduct:int, 
    Issue:int, 
    Subissue:int, 
    State:int,
    Company:int,
    Companyresponse:int, 
    timely:int, 
    weekday:int
    ):

    """
    
    """

    if timely is None:
        timely = int(predict_timely(Issue=Issue,Subissue=Subissue , Companyresponse=Companyresponse, Product=Product, Subproduct=Subproduct , State=State )["response01"])
    print(timely)

    
    features=pd.DataFrame({

            "Product":[Product],
            "Sub-product": [Subproduct],	
            "Issue":[Issue],	
            "Sub-issue":[Subissue],	
            "State":[State],	
            "Company response":[Companyresponse],
            "Timely response?": [timely],
            "Company":[Company],
            
            
            "weekday":[weekday],})
    


    features_pro=preprocesador.transform(features)

    #pred=modelo_dispute.predict(features_pro)[0]
    pred=(modelo_dispute.predict(features_pro) >= 0.23).astype(int)
    prob=modelo_dispute.predict(features_pro)

    return {"response": pred, "prob": prob}

In [13]:
product_encoder.transform(pd.DataFrame(["Debt collection"]))

c:\Users\unaif\anaconda3\envs\projecto\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


array([4])

In [14]:
predict_dispute("Debt collection"	,"Unknown or not specified"	,"Communication tactics",	"Frequent or repeated calls","TX", 			 	"In progress","Yes", "Premium Asset Services, LLC", "Monday")

Premium Asset Services, LLC
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step


C:\Users\unaif\AppData\Local\Temp\ipykernel_26896\2052658597.py:43: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return {"response": int(pred)}


{'response': 0}

In [21]:
iface = gr.Interface(
    fn=predict_dispute,
    inputs=[
        gr.Dropdown(choices=product_encoder.classes_.tolist(), value='Unknown or not specified', label="Product"),
        gr.Dropdown(choices=sub_product_encoder.classes_.tolist(), value='Unknown or not specified', label="Sub-product"),
        gr.Dropdown(choices=Issue_enc.classes_.tolist(), value='Unknown or not specified', label="Issue"),
        gr.Dropdown(choices=sub_Issue_enc.classes_.tolist(), value='Unknown or not specified', label="Sub-issue"),
        gr.Dropdown(choices=State_enc.classes_.tolist(), value='Unknown or not specified', label="State"),
    
        gr.Dropdown(choices=Company_response_enc.classes_.tolist(), value='Unknown or not specified', label="Company response"),
        
        gr.Dropdown(choices=["Yes", "No", None], value=None, label="Timely response?"),
        gr.Dropdown(choices=Company_enc.classes_.tolist(), value='Unknown or not specified', label="Company"),
        
        gr.Dropdown(choices=week_enc.classes_.tolist(),  label="weekday")
    ],
    outputs="json",
    title="Predicción Timely",
    description="Introduce los parámetros para obtener la predicción."
)
if __name__ == "__main__":
    iface.launch()

c:\Users\unaif\anaconda3\envs\projecto\Lib\site-packages\gradio\components\dropdown.py:230: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include: Unknown or not specified or set allow_custom_value=True.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\unaif\anaconda3\envs\projecto\Lib\site-packages\gradio\queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\unaif\anaconda3\envs\projecto\Lib\site-packages\gradio\route_utils.py", line 350, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\unaif\anaconda3\envs\projecto\Lib\site-packages\gradio\blocks.py", line 2235, in process_api
    inputs = await self.preprocess_data(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\unaif\anaconda3\envs\projecto\Lib\site-packages\gradio\blocks.py", line 1897, in preprocess_data
    processed_input.append(block.preprocess(inputs_cached))
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\unaif\anaconda3\envs\projecto\Lib\site-packages\gradio\components\dropdown.py", line 206, 

24 Asset Management Corp
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


In [16]:
"False "

'False '

24 Asset Management Corp
